In [180]:
import os
import pandas as pd
import glob
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
import concurrent.futures
from pycaret.classification import *
from sklearn.metrics import accuracy_score

In [134]:
open_file = open('model_columns.pkl', "rb")
model_columns = list(pickle.load(open_file))
open_file.close()

In [140]:
scraped_cols = ['Total shots','Expected goals (xG)','Offsides','Accurate passes percentage','score','Corners','Fouls committed','Shots on target','Blocks']
pred_cols = dict(zip(model_columns, scraped_cols))
model = load_model('lr_model')

Transformation Pipeline and Model Successfully Loaded


In [136]:
pred_cols

{'Shot Attempts': 'Total shots',
 'Total xG': 'Expected goals (xG)',
 'Offsides': 'Offsides',
 'Pass Completion': 'Accurate passes percentage',
 'Goals': 'score',
 'Corners': 'Corners',
 'Fouls Committed': 'Fouls committed',
 'Total Shots on Target': 'Shots on target',
 'Shots Blocked': 'Blocks'}

In [101]:
from os import walk

filenames = next(walk('C:/Users\idotz/OneDrive - Northeastern University/Python/Soccer Analysis'), (None, None, []))[2]
filenames = [f for f in filenames if '_data.pkl' in f]

In [102]:
filenames = ['20220826_data.pkl','20220828_data.pkl', '20220829_data.pkl', 'data.pkl']

In [103]:
all_data = []
for f in filenames:
    open_file = open(f, "rb")
    d = pickle.load(open_file)
    all_data.append(d)
    open_file.close()
    
all_data = [i for j in all_data for i in j]

In [104]:
f = []
for dic in all_data:
    d = {}
    for k, v in dic.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    f.append(d)


In [161]:
df = pd.DataFrame(f).iloc[:,:46]
df = df[df['half_time'] == 'Half-Time']

In [110]:
def score(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(1)
    driver.get(url)
    score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
    score = int(score[0]) - int(score[2])
    driver.quit()
    return score

In [111]:
score('https://www.fotmob.com/match/3888550/matchfacts/puebla-vs-fc-juarez')

0

In [124]:
scores = []
for i in range(0, len(df['url']), 2):
    x = list(df['url'][i:i + 2])
    print(x)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        r = executor.map(score, x)
    for result in r:
        scores.append(result)

['https://www.fotmob.com/match/3888550/matchfacts/puebla-vs-fc-juarez', 'https://www.fotmob.com/match/3888551/matchfacts/mazatlan-fc-vs-cf-america']
['https://www.fotmob.com/match/3900373/matchfacts/fc-utrecht-vs-ajax', 'https://www.fotmob.com/match/3904422/matchfacts/nantes-vs-toulouse']
['https://www.fotmob.com/match/3902828/matchfacts/dundee-united-vs-celtic', 'https://www.fotmob.com/match/3916965/matchfacts/gent-vs-royal-antwerp']
['https://www.fotmob.com/match/3887414/matchfacts/randers-fc-vs-aalborg-bk', 'https://www.fotmob.com/match/3887416/matchfacts/viborg-vs-lyngby']
['https://www.fotmob.com/match/3905446/matchfacts/lugano-vs-st.-gallen', 'https://www.fotmob.com/match/3900374/matchfacts/excelsior-vs-psv-eindhoven']
['https://www.fotmob.com/match/3900375/matchfacts/fc-volendam-vs-fc-twente', 'https://www.fotmob.com/match/3900963/matchfacts/aston-villa-vs-west-ham-united']
['https://www.fotmob.com/match/3900971/matchfacts/wolverhampton-wanderers-vs-newcastle-united', 'https://w

In [164]:
c = {v:k for k, v in pred_cols.items()}

In [165]:
c

{'Total shots': 'Shot Attempts',
 'Expected goals (xG)': 'Total xG',
 'Offsides': 'Offsides',
 'Accurate passes percentage': 'Pass Completion',
 'score': 'Goals',
 'Corners': 'Corners',
 'Fouls committed': 'Fouls Committed',
 'Shots on target': 'Total Shots on Target',
 'Blocks': 'Shots Blocked'}

In [166]:
df = df.rename(columns = c)

In [130]:
def winner(x):
    if x == 0:
        return 0
    elif x > 0:
        return 1
    else:
        return 2

In [174]:
pred_cols

{'Shot Attempts': 'Total shots',
 'Total xG': 'Expected goals (xG)',
 'Offsides': 'Offsides',
 'Pass Completion': 'Accurate passes percentage',
 'Goals': 'score',
 'Corners': 'Corners',
 'Fouls Committed': 'Fouls committed',
 'Total Shots on Target': 'Shots on target',
 'Shots Blocked': 'Blocks'}

In [175]:
df = df[list(c.values())]

In [178]:
df['pred'] = model[-1].predict(df)
df['final_score'] = scores
df['final_score'] = df['final_score'].apply(winner)

In [184]:
accuracy_score(df['pred'], df['final_score'])

0.6111111111111112

In [182]:
df

,Shot Attempts,Total xG,Offsides,Pass Completion,Goals,Corners,Fouls Committed,Total Shots on Target,Shots Blocked,pred,final_score
1,7.0,0.89,1.0,0.06,0.0,1.0,-3.0,0.0,-3.0,1,0
3,-5.0,-1.21,-1.0,-0.09,-1.0,-1.0,9.0,-2.0,2.0,2,2
4,-3.0,-0.74,0.0,-0.11,-2.0,-1.0,-3.0,-1.0,1.0,2,2
5,4.0,0.37,-1.0,0.04,-1.0,-1.0,-1.0,2.0,1.0,2,1
6,-12.0,-2.94,-1.0,-0.33,-4.0,-4.0,-1.0,-6.0,6.0,2,2
7,1.0,-0.95,1.0,-0.02,-1.0,2.0,0.0,-1.0,0.0,2,2
8,-3.0,-0.33,3.0,-0.15,1.0,-3.0,-1.0,0.0,3.0,1,1
9,4.0,-0.32,1.0,0.03,0.0,0.0,5.0,-1.0,-3.0,2,1
10,-7.0,-0.04,2.0,-0.16,-1.0,-2.0,1.0,-2.0,3.0,2,2
11,-6.0,-0.21,0.0,-0.10,-3.0,-3.0,0.0,-4.0,0.0,2,2
